# Assignment 5 Cheng Ye cy2696

### Install PySpark (Python Spark API)

In [ ]:
!pip install -U pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285397 sha256=7a49d288f0de3bee23d15bd151c536710818c7edeb930479ffd4ebc1fa38bbc4
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


### Loading Initial Libraries for later use

In [ ]:
import os
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

print("Using Apache Spark Version", spark.version)

Using Apache Spark Version 3.4.1


### Read a large CSV file into a Spark dataframe

In [ ]:
cb_sdf = spark.read.format("csv") \
               .options(header='true', inferschema='true', treatEmptyValuesAsNulls='true') \
               .load("/content/crunchbase_odm_orgs.csv")
cb_sdf.count()

1127735

## Implement PySpark code using DataFrames, RDDs or Spark UDF functions to answer questions

 ### Question1 : Find all companies with the name that is only two words (e.g. : "Goldman Sachs")
 print the count of such companies and show()only the name and location (city, region, country_code) in the resulting Spark DataFrame

In [ ]:
cb_rdd = cb_sdf.select('*').rdd.map(lambda row: [str(row[i]) for i in ['name','city','region','country_code']])

cb_sdf2 = spark.createDataFrame(cb_rdd,['name','city','region','country_code'])
import pyspark.sql.functions as F
companies_2 = cb_sdf2.where(F.size(F.split(F.col("name"), " ")) == 2)

In [ ]:
print("Count of companies with names that are only two words: ",companies_2.count()) # showing all companies with the name that is only two words (e.g. : "Goldman Sachs")

Count of companies with names that are only two words:  362534


In [ ]:
companies_2.show() # showing only the name and location (city, region, country_code) in the resulting Spark DataFrame

+--------------------+----------------+----------+------------+
|                name|            city|    region|country_code|
+--------------------+----------------+----------+------------+
|         Time Warner|        New York|  New York|         USA|
|       Goldman Sachs|        New York|  New York|         USA|
|     Jingle Networks|        New York|  New York|         USA|
|Hearst Communicat...|        New York|  New York|         USA|
|    Ning Interactive|      Menlo Park|California|         USA|
| Prosper Marketplace|   San Francisco|California|         USA|
|       Tribune Media|         Chicago|  Illinois|         USA|
| Aggregate Knowledge|       San Mateo|California|         USA|
|        Zing Systems|   Mountain View|California|         USA|
|         Amie Street|Long Island City|  New York|         USA|
|          Legg Mason|       Baltimore|  Maryland|         USA|
|        Haute Secure|         Seattle|Washington|         USA|
|          Squid Labs|         Alameda|C

### Question2 : Find all companies located in the state of California:
print the count of such companies and show()only the name and location (city, region, country_code) in the resulting Spark DataFrame


In [ ]:
import pyspark.sql.functions as F
companies_ca = cb_sdf2.where(F.col('region') == "California")

In [ ]:
print("Count of companies with names that are located in the state of California: ",companies_ca.count()) # showing all companies located in the state of California

Count of companies with names that are located in the state of California:  94871


In [ ]:
companies_ca.show()

+--------------------+--------------+----------+------------+
|                name|          city|    region|country_code|
+--------------------+--------------+----------+------------+
|                Zoho|    Pleasanton|California|         USA|
|            Facebook|    Menlo Park|California|         USA|
|               Accel|     Palo Alto|California|         USA|
|           Omnidrive|     Palo Alto|California|         USA|
|                Geni|West Hollywood|California|         USA|
|             Flektor|   Culver City|California|         USA|
|Fox Interactive M...| Beverly Hills|California|         USA|
|             Twitter| San Francisco|California|         USA|
|         StumbleUpon| San Francisco|California|         USA|
|              Scribd| San Francisco|California|         USA|
|             Slacker|     San Diego|California|         USA|
|                Lala|     Palo Alto|California|         USA|
|               Helio|   Los Angeles|California|         USA|
|       

### Question3 : Add a "Blog" column to the DataFrame with the row entries set to 1 if the "domain" field contains "blogspot.com", and 0 otherwise.
show() only the name, location (city, region, country_code) and "Blog" column for companies with the "Blog" field marked as 1

In [ ]:
cb_rdd = cb_sdf.select('*').rdd.map(lambda row: [str(row[i]) for i in ['name','domain','city','region','country_code']])

cb_sdf3 = spark.createDataFrame(cb_rdd,['name','domain','city','region','country_code'])

blog_add = cb_sdf3.select("*").withColumn("Blog", F.when(F.col("domain").contains("blogspot.com"), 1).otherwise(0))

companies_blog = blog_add.select('name','city','region','country_code','Blog')

comapnies_blog = companies_blog.where(F.col('Blog') == 1)

In [ ]:
print("Count of companies with 'domain' field contains blogspot.com: ",comapnies_blog.count()) # showing all companies with blogspot.com in their domain

Count of companies with 'domain' field contains blogspot.com:  394


In [ ]:
comapnies_blog.show()

+--------------------+-------------+------------+------------+----+
|                name|         city|      region|country_code|Blog|
+--------------------+-------------+------------+------------+----+
|     Sad Urdu Poetry|  San Antonio|       Texas|         USA|   1|
|      The Tech-Freak|    Sheffield|   Sheffield|         GBR|   1|
|           Ma.Gnolia|San Francisco|  California|         USA|   1|
|      Dynasty Online|         None|        None|        None|   1|
|            Hire-seo|         None|        None|        None|   1|
|          YelloYello|     Rijswijk|Zuid-Holland|         NLD|   1|
|       Youtubehiphop|    São Paulo|   Sao Paulo|         BRA|   1|
|     Payday advances|         None|        None|        None|   1|
|Blog Traffic Exch...|   Menlo Park|  California|         USA|   1|
|Sirius Forex Trad...|         None|        None|        None|   1|
|          Utilsforge|     Delaware|        Ohio|         USA|   1|
|      Discover India|    Faridabad|     Haryana

### Question4 : Find all companies with names that are palindromes (name reads the same way forward and reverse, e.g. madam) using Spark UDF function:
print the count and show() only the name and location (city, region, country_code) in the resulting Spark DataFrame

In [ ]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf
import pyspark.sql.functions as F

def is_palindrome(s):
    if s is None:
        return False
    return s.lower() == s[::-1].lower()

spark_udf = udf(is_palindrome, StringType())

cb_sdf4 = cb_sdf.withColumn('palindrome',spark_udf('name'))

companies = cb_sdf4['name','palindrome','city','region','country_code']

companies_pali = companies.filter(F.col('palindrome') == 'true')

companies_pali = companies_pali['name','city','region','country_code']




In [ ]:
print("Count of companies with names that are palindromes: ", companies_pali.count()) #showing all companies with names that are palindromes

Count of companies with names that are palindromes:  1132


In [ ]:
companies_pali.show()

+-------+--------------------+--------------------+------------+
|   name|                city|              region|country_code|
+-------+--------------------+--------------------+------------+
| Revver|         Los Angeles|          California|         USA|
|  KAYAK|            Stamford|         Connecticut|         USA|
|  ooVoo|            New York|            New York|         USA|
|Imikimi|             Seattle|          Washington|         USA|
|   Teet|        Lva Tolstogo|              Kaluga|         RUS|
| Mollom|Berchem-sainte-ag...|Brussels Hoofdste...|         BEL|
|  63336|              London|             England|         GBR|
|  TipiT|               Delft|        Zuid-Holland|         NLD|
|  beweb|            Auckland|            Auckland|         NZL|
|    CSC|        Falls Church|            Virginia|         USA|
|    CBC|              Ottawa|             Ontario|         CAN|
|  Orbro|              Howell|            Michigan|         USA|
|    OQO|       San Franc